# 中文分词
```
中文分词(Chinese Word Segmentation)指的是将一个汉字序列切分成一个个单独的词。分词就是将连续的字序列按照一定的规范重新组合成词序列的过程。
```

## 1. 难点
1. 词的定义，目前没有权威的规范。
2. 歧义问题。
3. 未登录词。

## 2. 基于词典的分词方法(机械分词)

这种方法本质上就是字符串匹配的方法，将一串文本中的文字片段和已有的词典进行匹配，如果匹配到，则此文字片段就作为一个分词结果。但是基于词典的机械切分会遇到多种问题，最为常见的包括`歧义切分`问题和`未登录词`问题。

常用的字符串匹配方法包括：`正向最大匹配法`（从左到右的方向）；`逆向最大匹配法`（从右到左的方向）；`最小切分`（每一句中切出的词数最小）`双向最大匹配`（进行从左到右、从右到左两次扫描）。

- 优点：速度快，时间复杂度可以保持在O（n）,实现简单
- 缺点：效果一般，对歧义和未登录词处理效果不佳。

### 2.1 简单最大匹配实现

In [2]:
word_dict = ['计算语言学', '课程', '意思']
max_len = 5
sentence = '计算语言学课程有意思'

copy = sentence
result = []
while copy:
    t = copy[-max_len:]
    while True:
        if t in word_dict or len(t) == 1:
            result.insert(0, t)
            copy = copy[:-len(t)]
            break
        else:
            t = t[1:]
            
print("/ ".join(result))

计算语言学/ 课程/ 有/ 意思


In [4]:
word_dict = ['计算语言学', '课程', '意思']
max_len = 5
sentence = '计算语言学课程有意思'

copy = sentence
result = []
while copy:
    t = copy[:max_len]
    while True:
        if t in word_dict or len(t) == 1:
            result.append(t)
            copy = copy[len(t):]
            break
        else:
            t = t[:-1]
            
print("/ ".join(result))

计算语言学/ 课程/ 有/ 意思


### 2.2 最大匹配法拓展 - MMSEG

#### 2.2.1 词典
1. 单字，给出每个字的词频。
2. 词语。

#### 2.2.2 匹配算法
1. 简单匹配:对于字符串中的汉字 $C_n$，用词典匹配以 $C_n$ 开头的子串并查找所有可能的匹配；
2. 复杂匹配:对于字符串中的汉字 $C_n$，查找所有可能以 $C_n$ 开头的`three-word chunks`，无论第一个汉语词是否有歧义。

#### 2.2.3 歧义消解规则
1. 最大匹配 

> The most plausible segmentation is the three-word chunk with maximum length.

在对句子中的某个词进行切分时，如果有歧义拿不定主意，就再向后展望两个汉语词，并且找出所有可能的“三词语块”。采纳候选中字长最长的“三词语块”的首词划分。

```
1.眼看 就要 来了（6）
2.眼看 就要 来（5）
3.眼看 就 要(4)
4.眼 看 就要(4)
5.眼 看 就(3)
```

2. 最大平均词长

> It is more likely to encounter multi-character words than one-character words.

```
1.国际化（平均词长=3）
2.国际 化（平均词长=1.5）
3.国 际 化（平均词长=1）
```

3. 最小词长方差

> Word lengths are usually evenly distributed.

```
1.研究 生命 起源 (0)
2.研究生 命 起源 (2/3)
```

4. 最大单字词语语素自由度之和

> 在语料库中出现频率高的汉字就很可能是一个单字词语，反之可能性就小。

计算单词词语语素自由度之和的公式是对“三词语块”中的单字词语频率取对数并求和。采纳其中和最大的语块的首词划分。

## 3. 基于统计的分词方法

目前中文分词的主流方法。利用统计机器学习模型学习词语切分的规律（称为训练），从而实现对未知文本的切分。

主要的统计模型有：N元文法模型（N-gram），隐马尔可夫模型（Hidden Markov Model ，HMM），最大熵模型（ME），条件随机场模型（Conditional Random Fields，CRF）等。

## 4. 基于深度学习的分词方法

近几年，深度学习方法为分词技术带来了新的思路，直接以最基本的向量化原子特征作为输入，经过多层非线性变换，输出层就可以很好的预测当前字的标记或下一个动作。在深度学习的框架下，仍然可以采用基于子序列标注的方式，或基于转移的方式，以及半马尔科夫条件随机场。

这类方法首先对语料的字进行嵌入，得到字嵌入后，将字嵌入特征输入给双向LSTM，输出层输出深度学习所学习到的特征，并输入给CRF层，得到最终模型。现有的方法包括：LSTM+CRF、BiLSTM+CRF等。

## 5. 基于理解的分词方法

基于理解的分词方法是通过让计算机模拟人对句子的理解，达到识别词的效果。其基本思想就是在分词的同时进行句法、语义分析，利用句法信息和语义信息来处理歧义现象。

# Jieba 模块

“结巴”中文分词：做最好的 Python 中文分词组件。

Github地址：https://github.com/fxsjy/jieba

- 基于前缀词典实现高效的词图扫描，生成句子中汉字所有可能成词情况所构成的有向无环图 (DAG)
- 采用了动态规划查找最大概率路径, 找出基于词频的最大切分组合
- 对于未登录词，采用了基于汉字成词能力的 HMM 模型，使用了 Viterbi 算法

In [10]:
!pip install jieba

In [4]:
import jieba

jieba.initialize()

r = jieba.lcut("孙悟空着肚子去上海洋人防工程的课")
print(r)

['孙悟空', '着', '肚子', '去', '上', '海洋', '人防', '工程', '的', '课']


## 函数原型
```python
def cut(self, sentence, cut_all=False, HMM=True):
   pass

def cut_for_search(self, sentence, HMM=True):
    pass

def lcut(self, sentence, cut_all=False, HMM=True):
   pass

def lcut_for_search(self, sentence, HMM=True):
    pass

```

## 作业

In [8]:
import jieba
import re
import time

t = time.time()
re_han = re.compile("[\u4E00-\u9FD5a-zA-Z0-9]+", re.U)

result = {}
with open('assert/三体.txt') as text:
    content = text.read()
c = jieba.cut(content)
for word in c:
    if not re_han.match(word):
        continue
    if not result.get(word):
        result[word] = 0
    result[word]+=1
    
result_l = list(result.items())
result_l.sort(key=lambda x:x[1], reverse=True)
t = time.time() - t
print('Finished. Time: %.1f s' % t)

Finished. Time: 4.5 s


In [3]:
from functools import reduce

words = [x for x in result_l if len(x[0])>1]
total = reduce(lambda x, y:x + y[1], words, 0)
for i in range(10):
    print(*words[i], '%.2f %%' % (words[i][1] / total))

一个 3088 0.01 %
没有 2141 0.01 %
他们 1691 0.01 %
我们 1559 0.01 %
自己 1376 0.00 %
这个 1358 0.00 %
程心 1347 0.00 %
已经 1284 0.00 %
现在 1276 0.00 %
世界 1250 0.00 %


## 源码调试

### 懒加载初始化字典

`self.FREQ`

```
<class 'dict'>: {'AT&T': 3, 'A': 0, 'AT': 0, 'AT&': 0, 'B超': 3, 'B': 0, 'c#': 3, 'c': 0, 'C#': 3, 'C': 0, 'c++': 3, 'c+': 0, 'C++': 3, 'C+': 0, 'T恤': 4, 'T': 0, 'A座': 3, 'A股': 3, 'A型': 3, 'A轮': 3, 'AA制': 3, 'AA': 0, 'AB型': 3, 'AB': 0, 'B座': 3, 'B股': 3, 'B型': 3, 'B轮': 3, 'BB机': 3, 'BB': 0, 'BP机': 3, 'BP': 0, 'C盘': 3, 'C座': 3, 'C语言': 3, 'C语': 0, 'CD盒': 3, 'CD': 0, 'CD机': 3, 'CALL机': 3, 'CA': 0, 'CAL': 0, 'CALL': 0, 'D盘': 3, 'D': 0, 'D座': 3, 'D版': 3, 'E盘': 3, 'E': 0, 'E座': 3, 'E化': 3, 'E通': 3, 'F盘': 3, 'F': 0, 'F座': 3, 'G盘': 3, 'G': 0, 'H盘': 3, 'H': 0, 'H股': 3, 'I盘': 3, 'I': 0, 'IC卡': 3, 'IC': 0, 'IP卡': 3, 'IP': 0, 'IP电话': 3, 'IP电': 0, 'IP地址': 3, 'IP地': 0, 'K党': 3, 'K': 0, 'K歌之王': 3, 'K歌': 0, 'K歌之': 0, 'N年': 3, 'N': 0, 'O型': 3, 'O': 0, 'PC机': 3, 'P': 0, 'PC': 0, 'PH值': 3, 'PH': 0, 'SIM卡': 3, 'S': 0, 'SI': 0, 'SIM': 0, 'U盘': 3, 'U': 0, 'VISA卡': 3, 'V': 0, 'VI': 0, 'VIS': 0, 'VISA': 0, 'Z盘': 3, 'Z': 0, 'Q版': 3, 'Q': 0, 'QQ号': 3...
```

```python
REQ["清"] = 17519
REQ["清华"] = 1057
REQ["清华大"] = 0
REQ["清华大学"] = 922
```

### 生成DAG

In [2]:
def get_DAG(self, sentence):
    self.check_initialized()
    DAG = {}
    N = len(sentence)
    for k in xrange(N):
        tmplist = []
        i = k
        frag = sentence[k]
        while i < N and frag in self.FREQ:
            if self.FREQ[frag]:
                tmplist.append(i)
            i += 1
            frag = sentence[k:i + 1]
        if not tmplist:
            tmplist.append(k)
        DAG[k] = tmplist
    return DAG

'孙悟空着肚子去上海洋人防工程的课'

<class 'dict'>: {0: [0, 2], 1: [1, 2], 2: [2], 3: [3], 4: [4, 5], 5: [5], 6: [6], 7: [7, 8], 8: [8, 9], 9: [9, 10], 10: [10, 11], 11: [11], 12: [12, 13], 13: [13], 14: [14], 15: [15]}

### 全模式

In [ ]:
def __cut_all(self, sentence):
    dag = self.get_DAG(sentence)
    old_j = -1
    for k, L in iteritems(dag):
        if len(L) == 1 and k > old_j:
            yield sentence[k:L[0] + 1]
            old_j = L[0]
        else:
            for j in L:
                if j > k:
                    yield sentence[k:j + 1]
                    old_j = j

### 精确无 HMM

In [3]:
def calc(self, sentence, DAG, route):
    N = len(sentence)
    route[N] = (0, 0)
    logtotal = log(self.total)
    for idx in xrange(N - 1, -1, -1):
        route[idx] = max((log(self.FREQ.get(sentence[idx:x + 1]) or 1)
                -
                    logtotal + route[x + 1][0], x) for x in DAG[idx])

In [ ]:
def __cut_DAG_NO_HMM(self, sentence):
    DAG = self.get_DAG(sentence)
    route = {}
    self.calc(sentence, DAG, route)
    x = 0
    N = len(sentence)
    buf = ''
    while x < N:
        y = route[x][1] + 1
        l_word = sentence[x:y]
        if re_eng.match(l_word) and len(l_word) == 1:
            buf += l_word
            x = y
        else:
            if buf:
                yield buf
                buf = ''
            yield l_word
            x = y
    if buf:
        yield buf
        buf = ''

### HMM

维特比算法

> 来源主要有两个： 一个是网上能下载到的1998人民日报的切分语料还有一个msr的切分语料; 另一个是作者自己收集的一些txt小说，用ictclas把他们切分（可能有一定误差）。 然后用python脚本统计词频

#### 初始概率分布
```
P={'B': -0.26268660809250016,
 'E': -3.14e+100,
 'M': -3.14e+100,
 'S': -1.4652633398537678}```

#### 状态转移概率分布
```
P={'B': {'E': -0.510825623765990, 'M': -0.916290731874155},
 'E': {'B': -0.5897149736854513, 'S': -0.8085250474669937},
 'M': {'E': -0.33344856811948514, 'M': -1.2603623820268226},
 'S': {'B': -0.7211965654669841, 'S': -0.6658631448798212}}```

#### 观测概率
```
P={'B': {'\u4e00': -3.6544978750449433,
       '\u4e01': -8.125041941842026,
       '\u4e03': -7.817392401429855,
        ...
   'E': {'\u4e00': -3.6544978750449433,
       '\u4e01': -8.125041941842026,
       '\u4e03': -7.817392401429855,
       ...```
        

In [5]:
def __cut(sentence):
    global emit_P
    prob, pos_list = viterbi(sentence, 'BMES', start_P, trans_P, emit_P)
    begin, nexti = 0, 0
    # print pos_list, sentence
    for i, char in enumerate(sentence):
        pos = pos_list[i]
        if pos == 'B':
            begin = i
        elif pos == 'E':
            yield sentence[begin:i + 1]
            nexti = i + 1
        elif pos == 'S':
            yield char
            nexti = i + 1
    if nexti < len(sentence):
        yield sentence[nexti:]